In [26]:
# https://github.com/helgefmi/ohno/blob/master/ohno/client/pty.py ## loosely based on this code
import os
import pty
import subprocess
import pprint
import time
import json
import select
import re
class Terminal:
    # This pattern will capture everything I have seen so far.  It assumes that you'll never have one that ends with a digit followed immediately by an actual letter
    # it also theoretically allows for certain kinds of garbage that should never actually show up in a real terminal escape sequence
    escape_pattern = re.compile("^\x1b\[?(\??\)?[\d;]*)(\w)")

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.reset()

    def reset(self):
        self.cursor_row = 0
        self.cursor_col = 0
        self.characters = [[' ' for i in range(self.cols)] for _ in range(self.rows)]
        self.foregrounds = [["white" for i in range(self.cols)] for _ in range(self.rows)]
        self.backgrounds = [["black" for i in range(self.cols)] for _ in range(self.rows)] # does Nethack actually do this?

    def text(self):
        s = ""
        for i in range(self.rows):
            for j in range(self.cols): # let's ignore colors for now
                s += self.characters[i][j]
            s += "\n"
        return s
    
    def print(self):
        print(self.text())
                
    def update(self, input, print_instructions = False):
        #print_instructions = True # for now
        #self.reset()
        # this is where the logic is going to get complex
        instructions = []
        while input:
            ch = input[0]
            if ch == "\x1b":
                # find the first group that matches the escape pattern
                m = Terminal.escape_pattern.match(input)
                if m:
                    #print(f"found match beginning with {input[:20]}")
                    code = "\x1b" + m.group(1) + m.group(2)
                    instructions.append(code)
                    #print(f"length of code is {len(code)}")
                    #input = input[len(code):]
                    input = input[m.end():]
                    #print(f"found escape sequence \\x1b{code[1:]}")
                    #print(f"next character is {input[0]}")
                else:
                    print("uh oh, couldn't match the escape sequence")
                    print(input[:20])
            else:
                instructions.append(ch)
                input = input[1:]

        for ch in instructions:
            if len(ch) == 1:
                # not an escape sequence
                if ch == "\r": # carriage return; reset cursor column to 0
                    self.cursor_col = 0 # carriage return; reset cursor column to 0
                elif ch == "\n": # newline; move cursor row down by one
                    self.cursor_row += 1
                elif ch == "\b": # backspace; move cursor column back by one
                    self.cursor_col -= 1  # backspace; move cursor column back by one
                elif ch == '\x0f' or ch == '\x00': # some kind of no-op; not sure why they're here
                    pass
                else: # normal, displayable character
                    self.characters[self.cursor_row][self.cursor_col] = ch
                    self.cursor_col += 1
            else:
                escape, args, last_char = ch[0], ch[1:-1], ch[-1]
                if ch in ("\x1b7", "\x1b8", "\x1b?47h"):
                    print(f"ignored unhandled save/restore sequence \\x1b{ch[1:]}")
                    continue
                elif last_char.isdigit():
                    # I don't know what to do with these yet
                    print(f"unknown unhandled escape sequence \\x1b{ch[1:]}")
                    continue
                
                if not args:
                    if last_char in 'ABCDEFSTf':
                        args = [1]
                    elif last_char == 'H':
                        args = [1, 1]
                    else:
                        args = [0]
                else:
                    try:
                        args = [int(x) for x in args.split(';')]
                    except ValueError:
                        raise ValueError(f"couldn't parse escape sequence \\x1b{ch[1:]}")

                if last_char == "H": # set cursor position
                    self.cursor_row = args[0] - 1
                    self.cursor_col = args[1] - 1

                elif last_char == "J": # clear (parts of) screen
                    if args[0] == 0: # clear to end of screen
                        for i in range(self.cursor_col, self.cols):
                            self.characters[self.cursor_row][i] = ' '
                        for i in range(self.cursor_row + 1, self.rows):
                            for j in range(self.cols):
                                self.characters[i][j] = ' '
                    elif args[0] == 1: # clear to beginning of screen
                        for i in range(self.cursor_col+1):
                            self.characters[self.cursor_row][i] = ' '
                        for i in range(self.cursor_row):
                            for j in range(self.cols):
                                self.characters[i][j] = ' '
                    elif args[0] == 2: # clear entire screen
                        for i in range(self.rows):
                            for j in range(self.cols):
                                self.characters[i][j] = ' '
                    else:
                        print(f"unhandled clear screen \\x1b{ch[1:]}")

                elif last_char == "K": # clear (parts of) line
                    if args[0] == 0:
                        for i in range(self.cursor_col, self.cols):
                            self.characters[self.cursor_row][i] = ' '
                    elif args[0] == 1:
                        for i in range(self.cursor_col+1):
                            self.characters[self.cursor_row][i] = ' '
                    elif args[0] == 2:
                        for i in range(self.cols):
                            self.characters[self.cursor_row][i] = ' '
                    else:
                        print(f"unhandled clear line \\x1b{ch[1:]}")

                elif last_char == "A": # move cursor up
                    self.cursor_row -= args[0]
                elif last_char == "B": # move cursor down
                    self.cursor_row += args[0]
                elif last_char == "C": # move cursor right
                    self.cursor_col += args[0]
                elif last_char == "D": # move cursor left
                    self.cursor_col -= args[0]
                
                elif last_char == "m":
                    #print(f"ignored color sequence")]
                    pass
                else:
                    print(f"unhandled escape sequence \\x1b{ch[1:]}")
           
        # this code is taken from the example; I'm not sure if it's necessary and I don't know if the opposite check (<0) also needs to be done
        while self.cursor_col >= self.cols:
            self.cursor_col = self.cursor_col - self.cols # wrap around to the next line
            self.cursor_row += 1
        while self.cursor_row >= self.rows:
            self.cursor_row = self.rows - 1 # don't go off the screen


class Nethack:
    def __init__(self):
        self.master = None
        self.slave = None
        self.process = None
        self.terminal = Terminal(24, 80)

    def start_process(self):
        self.master, self.slave = pty.openpty()
        self.process = subprocess.Popen(
            ["nethack"],
            stdin=self.slave,
            stdout=self.slave,
            stderr=self.slave,
            close_fds=True, # this line comes from Bing Chat and I don't know what it does really
            universal_newlines=True,
        )
        os.close(self.slave) # Copilot added this so take it with a grain of salt...if it works, we don't need to save the reference

    def read_output(self, timeout = 1.0, sleep = 0.25, chars = 8192):
        if self.process is None:
            raise ValueError("Process not started")
        if sleep is not None and sleep > 0:
            time.sleep(sleep)
        if timeout is not None and timeout > 0:
            rlist, _, _ = select.select([self.master], [], [], timeout)
            if rlist:
                return os.read(self.master, chars).decode()
            else:
                print("(read timed out)")
                print("(this usually means the input sent did not update the game state.)")
                return None
        else:
            return os.read(self.master, chars).decode()

    def send_command(self, command):
        #os.write(self.master, command.encode() + b"\n")
        os.write(self.master, command.encode())

    def close(self):
        try:
            self.process.terminate()
            os.close(self.master)
        except:
            pass

    def next_command(self, c, verbose = False, timeout = 1.0):
        self.send_command(c)
        text = self.read_output(timeout = timeout)
        if verbose:
            print(text)
        if text is not None:
            self.terminal.update(text, print_instructions = verbose)
        print(self.terminal.text())
        return text

    @staticmethod
    def clean_locks():
        password = json.load(open("private.json"))["sudo"]
        os.system(f'echo "{password}" | sudo -S rm -f /var/games/nethack/*lock*')

    @staticmethod
    def get_new_process(verbose = False):
        Nethack.clean_locks()
        n = Nethack()
        n.start_process()
        text = n.read_output(timeout = 1.0)
        n.terminal.update(text, print_instructions = verbose)
        print(n.terminal.text())
        return n

# 24, 80

In [11]:
ENTER = RETURN = "\x0d"
L = LEFT = "h"
R = RIGHT = "l"
U = UP = "k"
D = DOWN = "j"

In [27]:
nethack = Nethack.get_new_process()

[sudo] password for perplexity: 

ignored unhandled save/restore sequence \x1b7
ignored unhandled save/restore sequence \x1b?47h
unknown unhandled escape sequence \x1b)0
                                                                                
NetHack, Copyright 1985-2023                                                    
         By Stichting Mathematisch Centrum and M. Stephenson.                   
         Version 3.6.7 Unix, revised Apr  1 07:02:11 2024.                      
         See license for details.                                               
                                                                                
                                                                                
Shall I pick character's race, role, gender and alignment for you? [ynaq]       
                                                                                
                                                                                
                                                      

In [28]:
nethack.next_command("n")

       name: perplexity                  Pick a role or profession              
       role:  choosing now                                                      
       race:  not yet specified          a - an Archeologist                    
     gender:  not yet specified          b - a Barbarian                        
  alignment:  not yet specified          c - a Caveman/Cavewoman                
                                         h - a Healer                           
                                         k - a Knight                           
                                         m - a Monk                             
                                         p - a Priest/Priestess                 
                                         r - a Rogue                            
                                         R - a Ranger                           
                                         s - a Samurai                          
                            

'n\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role:  choosing now\r\x1b[3;1H       race:  not yet specified\r\x1b[4;1H     gender:  not yet specified\r\x1b[5;1H  alignment:  not yet specified\x1b[1;41H\x1b[K Pick a role or profession\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K a - an Archeologist\x1b[4;41H\x1b[K b - a Barbarian\x1b[5;41H\x1b[K c - a Caveman/Cavewoman\x1b[6;41H\x1b[K h - a Healer\x1b[7;41H\x1b[K k - a Knight\x1b[8;41H\x1b[K m - a Monk\x1b[9;41H\x1b[K p - a Priest/Priestess\x1b[10;41H\x1b[K r - a Rogue\x1b[11;41H\x1b[K R - a Ranger\x1b[12;41H\x1b[K s - a Samurai\x1b[13;41H\x1b[K t - a Tourist\x1b[14;41H\x1b[K v - a Valkyrie\x1b[15;41H\x1b[K w - a Wizard\x1b[16;41H\x1b[K * + Random\x1b[17;41H\x1b[K \x1b[18;41H\x1b[K / - Pick race first\x1b[19;41H\x1b[K " - Pick gender first\x1b[20;41H\x1b[K [ - Pick alignment first\x1b[21;41H\x1b[K q - Quit\x1b[22;41H\x1b[K\x1b[C(end) '

In [29]:
nethack.next_command("v")

       name: perplexity                  Pick a race or species                 
       role: Valkyrie                                                           
       race:  choosing now               h - human                              
     gender: female                      d - dwarf                              
  alignment:  not yet specified          * + Random                             
                                                                                
                                         ? - Pick another role first            
                                             role forces female                 
                                         [ - Pick alignment first               
                                         q - Quit                               
                                         (end)                                  
                                                                                
                            

'\x1b[14;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[13;40H\x1b[K\x1b[14;40H\x1b[K\x1b[15;40H\x1b[K\x1b[16;40H\x1b[K\x1b[17;40H\x1b[K\x1b[18;40H\x1b[K\x1b[19;40H\x1b[K\x1b[20;40H\x1b[K\x1b[21;40H\x1b[K\x1b[22;40H\x1b[K\x1b[23;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race:  choosing now\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment:  not yet specified\x1b[1;41H\x1b[K Pick a race or species\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K h - human\x1b[4;41H\x1b[K d - dwarf\x1b[5;41H\x1b[K * + Random\x1b[6;41H\x1b[K \x1b[7;41H\x1b[K ? - Pick another role first\x1b[8;41H\x1b[K     role forces female\x1b[9;41H\x1b[K [ - Pick alignment first\x1b[10;41H\x1b[K q - Quit\x1b[11;41H\x1b[K\x1b[C(end) '

In [30]:
nethack.next_command("h")

       name: perplexity                  Pick an alignment or creed             
       role: Valkyrie                                                           
       race: human                       l - lawful                             
     gender: female                      n - neutral                            
  alignment:  choosing now               * + Random                             
                                                                                
                                         ? - Pick another role first            
                                         / - Pick another race first            
                                             role forces female                 
                                         q - Quit                               
                                         (end)                                  
                                                                                
                            

'\x1b[3;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race: human\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment:  choosing now\x1b[1;41H\x1b[K Pick an alignment or creed\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K l - lawful\x1b[4;41H\x1b[K n - neutral\x1b[5;41H\x1b[K * + Random\x1b[6;41H\x1b[K \x1b[7;41H\x1b[K ? - Pick another role first\x1b[8;41H\x1b[K / - Pick another race first\x1b[9;41H\x1b[K     role forces female\x1b[10;41H\x1b[K q - Quit\x1b[11;41H\x1b[K\x1b[C(end) '

In [31]:
nethack.next_command("n")

       name: perplexity                  Is this ok? [ynq]                      
       role: Valkyrie                                                           
       race: human                       perplexity, neutral human Valkyrie     
     gender: female                                                             
  alignment: neutral                     y + Yes; start game                    
                                         n - No; choose role again              
                                         q - Quit                               
                                         (end)                                  
                                                                                
                                                                                
                                                                                
                                                                                
                            

'\x1b[4;44H+\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[10;40H\x1b[K\x1b[11;40H\x1b[K\x1b[12;40H\x1b[K\x1b[H\x1b[2J\x1b[H       name: perplexity\r\x1b[2;1H       role: Valkyrie\r\x1b[3;1H       race: human\r\x1b[4;1H     gender: female\r\x1b[5;1H  alignment: neutral\x1b[1;41H\x1b[K Is this ok? [ynq]\x1b[2;41H\x1b[K \x1b[3;41H\x1b[K perplexity, neutral human Valkyrie\x1b[4;41H\x1b[K \x1b[5;41H\x1b[K y + Yes; start game\x1b[6;41H\x1b[K n - No; choose role again\x1b[7;41H\x1b[K q - Quit\x1b[8;41H\x1b[K\x1b[C(end) '

In [32]:
nethack.next_command("y")

                       It is written in the Book of Odin:                       
                                                                                
                           After the Creation, the cruel god Moloch rebelled    
                           against the authority of Marduk the Creator.         
                           Moloch stole from Marduk the most powerful of all    
                           the artifacts of the gods, the Amulet of Yendor,     
                           and he hid it in the dark cavities of Gehennom, the  
                           Under World, where he now lurks, and bides his time. 
                                                                                
                       Your god Odin seeks to possess the Amulet, and with it   
                       to gain deserved ascendance over the other gods.         
                                                                                
                       You, 

'\x1b[5;44H-\x1b[1;40H\x1b[K\x1b[2;40H\x1b[K\x1b[3;40H\x1b[K\x1b[4;40H\x1b[K\x1b[5;40H\x1b[K\x1b[6;40H\x1b[K\x1b[7;40H\x1b[K\x1b[8;40H\x1b[K\x1b[9;40H\x1b[K\x1b[H\x1b[2J\x1b[H\x1b[H\x1b[2J\x1b[H\x1b[17;28H-----\x1b[m\x1b[C---------\x1b[m\x1b[18;28H|.........\x1b[m\x1bd\x1b[m..\x1b[m\x1b\x1bx\x1b[m|\x1b[m\x1b[19;28H|..........\x1b[m\x1b\x1b@\x1b[m..|\x1b[m\x1b[20;28H|.......\x1b[m\x1b)\x1b[m..\x1b[m\x1b\x1b\x1bd\x1b[m\x1b[m..\x1b[m\x1b-\x1b[m\x1b[21;28H---------------\x1b[m\x1b[19;39H\x1b[23;1H\x1b[K\x1b[24;1H\x1b[K\x1b[A[\x1bPerplexity the Stripling      \x1b[m] St:18/02 Dx:11 Co:15 In:8 Wi:10 Ch:10 Neutral\x1b[K\r\x1b[24;1HDlvl:1 $:0 HP:16(16) Pw:2(2) AC:6 Xp:1\x1b[K\x1b[1;23H\x1b[K It is written in the Book of Odin:\x1b[2;23H\x1b[K \x1b[3;23H\x1b[K     After the Creation, the cruel god Moloch rebelled\x1b[4;23H\x1b[K     against the authority of Marduk the Creator.\x1b[5;23H\x1b[K     Moloch stole from Marduk the most powerful of all\x1b[6;23H\x1b[K     the artifacts of the gods, the

In [35]:
RETURN = "\x0d"
LEFT = "h"
RIGHT = "l"
UP = "k"
DOWN = "j"
nethack.next_command(RETURN)

Be careful!  New moon tonight.                                                  
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                            

'\r\x1b[A\x1b[K\x1b[2;1H\x1b[K\x1b[H\x1b[HBe careful!  New moon tonight.\x1b[K\x1b[23;1H[\x1bPerplexity the Stripling      \x1b[m] St:18/02 Dx:11 Co:15 In:8 Wi:10 Ch:10 Neutral\r\x1b[24;1HDlvl:1 $:0 HP:16(16) Pw:2(2) AC:6 Xp:1\x1b[19;39H'

In [36]:
nethack.next_command(UP, timeout = 0.5)

'\x1b[H\x1b[K\x1b[18;39H\x1b\x1b@\x1b[m\x1b[19;39H<\x1b[m\x1b[A\x08'

In [51]:
nethack.next_command(R, timeout = 0.5)

'#\x1b[m\x1b\x1b@\x1b[m\x1b[18;46H#\x1b[m\x1b[A'

In [15]:
# h is left, j is down, k is up, l is right
#text = next_command("j")  # okay, something odd is happening here: When I move, the player cursor disappears, and then the right way moves to the left.  So it seems like some portion of the text is getting blotted out.
# interesting, that final character is \x08 which is backspace.  I wonder if that has smething to do with it.
nethack.next_command("k", timeout = 0.5, verbose = True)
# The problems I'm seeing here seem to be of a form where the I see the new locations of entities, but the old locations are not being cleared.
# It's actually a bit weirder than that.  There's some jumping around.  That kind of makes it seem like some of the control characters are not being handled correctly.
# Which sucks, because I know I already put a ton of work into that.  And I know it's a pain to debug.
# So, one question here is whether the things are not updating, or whether they're updating incorrectly.  That should actually not be too hard to figure out; we could just 
# clear the screen either to blanks or to a single character, and then see if the entities are being drawn correctly.
# So it looks to be a matter of not updating things that should be updating; however, we're also getting some strange things...
# ...the player did not get drawn after switching places with the pet.
# ...sometimes the player "teleports" to the top-center of the screen.
# When I get rid of the full clear, we do see some teleporting around.  You might expect that if the screen is getting re-centered, but then you would expect other things to happen as well.

d@...d[
SetCursor(move=CursorMove(x=0, y=-1, relative=True))

SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(3))
d
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=1, y=0, relative=True))
SetAttribute(attribute=<Attribute.BOLD: 1>)
SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(7))
@
SetAttribute(attribute=<Attribute.NORMAL: 0>)
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=16, y=65, relative=False))
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=1, y=0, relative=True))
SetCursor(move=CursorMove(x=1, y=0, relative=True))
.
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=CursorMove(x=17, y=68, relative=False))
SetAttribute(attribute=<Attribute.BOLD: 1>)
SetColor(role=<ColorRole.FOREGROUND: 30>, color=Ansi256(7))
SetAttribute(attribute=<Attribute.REVERSE: 7>)
d
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetAttribute(attribute=<Attribute.NORMAL: 0>)
SetCursor(move=Cursor

'\x1b[A\x08\x08\x1bd\x1b[m\x1b[C\x1b\x1b@\x1b[m.\x1b[m\x1b[17;66H.\x1b[m\x1b[C\x1b[C.\x1b[m\x1b[18;69H\x1b\x1b\x1bd\x1b[m\x1b[m\x1b[A\x1b[A\x08'